# Preprocessing of rental data, apartment sales data & postal code keys data

## Rental apartment data

##### Read .csv-file

In [1]:
import pandas as pd

csv_file = 'raw_data/13eb_vuokra-asunnot.csv'

rentals_raw_df = pd.read_csv(csv_file, delimiter=';', encoding='latin1')

rentals_raw_df.head()

,Vuosineljännes,Postinumero,Huoneluku,Lukumäärä,Neliövuokra (eur/m2)
0,2015Q1,00100 Helsinki Keskusta - Etu-Töölö (Helsinki ),Yksiöt,452,24.26
1,2015Q1,00100 Helsinki Keskusta - Etu-Töölö (Helsinki ),Kaksiot,243,18.36
2,2015Q1,00100 Helsinki Keskusta - Etu-Töölö (Helsinki ),Kolmiot+,34,13.09
3,2015Q1,00120 Punavuori (Helsinki ),Yksiöt,95,21.52
4,2015Q1,00120 Punavuori (Helsinki ),Kaksiot,52,14.77


In [2]:
rentals_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58548 entries, 0 to 58547
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Vuosineljännes        58548 non-null  object
 1   Postinumero           58548 non-null  object
 2   Huoneluku             58548 non-null  object
 3   Lukumäärä             58548 non-null  object
 4   Neliövuokra (eur/m2)  58548 non-null  object
dtypes: object(5)
memory usage: 2.2+ MB


##### Rename columns

In [3]:
rentals_raw_df.rename(
    columns={
        'Vuosineljännes': 'Year', 
        'Postinumero': 'Postal code',
        'Huoneluku': 'Rooms',
        'Lukumäärä': 'Rental apartment count',
        'Neliövuokra (eur/m2)': 'Rent per m2'
    },
    inplace=True
)

##### Remove rows with no data

In [4]:
filtered_rentals_df = rentals_raw_df[rentals_raw_df['Rental apartment count'] != '.'].reset_index(drop=True)

filtered_rentals_df.shape

(34341, 5)

##### Trim values in columns 'Year' & 'Postal code'

In [5]:
filtered_rentals_df['Year'] = filtered_rentals_df['Year'].str.slice(0, 4).astype(int)
filtered_rentals_df['Postal code'] = filtered_rentals_df['Postal code'].str.slice(0, 5).astype(int)

##### Categorize number of rooms

In [6]:
category_codes = {'Yksiöt': 1, 'Kaksiot': 2, 'Kolmiot+': 3}
filtered_rentals_df['Rooms'] = pd.Categorical(filtered_rentals_df['Rooms'], categories=category_codes.keys(), ordered=True)
filtered_rentals_df['Rooms'] = filtered_rentals_df['Rooms'].cat.rename_categories(category_codes)

##### Change types

In [7]:
filtered_rentals_df = filtered_rentals_df.astype({'Rental apartment count': int, 'Rent per m2': float})

filtered_rentals_df.head()

,Year,Postal code,Rooms,Rental apartment count,Rent per m2
0,2015,100,1,452,24.26
1,2015,100,2,243,18.36
2,2015,100,3,34,13.09
3,2015,120,1,95,21.52
4,2015,120,2,52,14.77


In [8]:
filtered_rentals_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34341 entries, 0 to 34340
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   Year                    34341 non-null  int64   
 1   Postal code             34341 non-null  int64   
 2   Rooms                   34341 non-null  category
 3   Rental apartment count  34341 non-null  int64   
 4   Rent per m2             34341 non-null  float64 
dtypes: category(1), float64(1), int64(3)
memory usage: 1.1 MB


##### Group quarters and count yearly apartment count & average rent

In [9]:
rentals_grouped_by = filtered_rentals_df.groupby(['Year', 'Postal code', 'Rooms'], as_index=False)
rentals_df = rentals_grouped_by.agg({
    'Year': 'first',
    'Postal code': 'first',
    'Rooms': 'first',
    'Rent per m2': 'mean',
    'Rental apartment count': 'sum'
})
rentals_df = rentals_df[rentals_df['Rental apartment count'] != 0].reset_index(drop=True)

rentals_df.head()

,Year,Postal code,Rooms,Rent per m2,Rental apartment count
0,2015.0,100.0,1,24.7325,1820
1,2015.0,100.0,2,18.3975,993
2,2015.0,100.0,3,14.1425,156
3,2015.0,120.0,1,22.6525,410
4,2015.0,120.0,2,14.8325,242


##### Reset types for 'Year' & 'Postal code'

In [10]:
rentals_df = rentals_df.astype({'Year': int, 'Postal code': int})
rentals_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9904 entries, 0 to 9903
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   Year                    9904 non-null   int64   
 1   Postal code             9904 non-null   int64   
 2   Rooms                   9904 non-null   category
 3   Rent per m2             9904 non-null   float64 
 4   Rental apartment count  9904 non-null   int64   
dtypes: category(1), float64(1), int64(3)
memory usage: 319.4 KB


##### Write to .csv

In [11]:
file_path = 'preprocessed_data/preprocessed_rental_apartments.csv'
rentals_df.to_csv(file_path, index=False)

## Apartment sales data

In [12]:
csv_file = 'raw_data/13mu_vanhat_osakeasunnot.csv'

sales_raw_df = pd.read_csv(csv_file, delimiter=';', encoding='latin1')

sales_raw_df.head()

,Vuosi,Postinumero,Talotyyppi,Neliöhinta (EUR/m2),"Kauppojen lukumäärä, varainsiirtoverotiedot vuodesta 2020 alkaen"
0,2009,00100 Helsinki keskusta - Etu-Töölö (Helsinki),Kerrostalo yksiöt,4817,116
1,2009,00100 Helsinki keskusta - Etu-Töölö (Helsinki),Kerrostalo kaksiot,4470,167
2,2009,00100 Helsinki keskusta - Etu-Töölö (Helsinki),Kerrostalo kolmiot+,4234,122
3,2009,00120 Punavuori - Bulevardi (Helsinki),Kerrostalo yksiöt,5040,51
4,2009,00120 Punavuori - Bulevardi (Helsinki),Kerrostalo kaksiot,5044,70


In [13]:
sales_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71778 entries, 0 to 71777
Data columns (total 5 columns):
 #   Column                                                            Non-Null Count  Dtype 
---  ------                                                            --------------  ----- 
 0   Vuosi                                                             71778 non-null  int64 
 1   Postinumero                                                       71778 non-null  object
 2   Talotyyppi                                                        71778 non-null  object
 3   Neliöhinta (EUR/m2)                                               71778 non-null  object
 4   Kauppojen lukumäärä, varainsiirtoverotiedot vuodesta 2020 alkaen  71778 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.7+ MB


##### Rename columns

In [14]:
sales_raw_df.rename(
    columns={
        'Vuosi': 'Year',
        'Postinumero': 'Postal code',
        'Talotyyppi': 'Rooms', 
        'Neliöhinta (EUR/m2)': 'Sales price per m2',
        'Kauppojen lukumäärä, varainsiirtoverotiedot vuodesta 2020 alkaen': 'Sold apartments count'
    }, 
    inplace=True
)

##### Remove rows with no data

In [15]:
filtered_sales_df = sales_raw_df[(sales_raw_df['Sales price per m2'] != '.') & (sales_raw_df['Sales price per m2'] != '...')].reset_index(drop=True)

filtered_sales_df.shape

(18233, 5)

##### Trim values in column 'Postal code'

In [16]:
filtered_sales_df['Postal code'] = filtered_sales_df['Postal code'].str.slice(0, 5).astype(int)

##### Categorize number of rooms

In [17]:
category_codes = {'Kerrostalo yksiöt': 1, 'Kerrostalo kaksiot': 2, 'Kerrostalo kolmiot+': 3}
filtered_sales_df['Rooms'] = pd.Categorical(filtered_sales_df['Rooms'], categories=category_codes.keys(), ordered=True)
filtered_sales_df['Rooms'] = filtered_sales_df['Rooms'].cat.rename_categories(category_codes)

##### Change types

In [18]:
sales_df = filtered_sales_df.astype({'Sales price per m2': int, 'Sold apartments count': int})

sales_df.head()

,Year,Postal code,Rooms,Sales price per m2,Sold apartments count
0,2009,100,1,4817,116
1,2009,100,2,4470,167
2,2009,100,3,4234,122
3,2009,120,1,5040,51
4,2009,120,2,5044,70


In [19]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18233 entries, 0 to 18232
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   Year                   18233 non-null  int64   
 1   Postal code            18233 non-null  int64   
 2   Rooms                  18233 non-null  category
 3   Sales price per m2     18233 non-null  int64   
 4   Sold apartments count  18233 non-null  int64   
dtypes: category(1), int64(4)
memory usage: 587.8 KB


##### Write to .csv

In [20]:
file_path = 'preprocessed_data/preprocessed_sold_apartments.csv'
sales_df.to_csv(file_path, index=False)

## Postal code keys data

##### Read .xlsx-file

In [21]:
postal_raw_df = pd.read_excel('raw_data/postinumerot.xlsx', skiprows=2)

postal_raw_df.head()

,Postinumeroalue,Postinumeroalueen nimi,Postinumeroalueen nimi (sv),Kunta,Kunnan nimi,Kuntaryhmä,Kuntaryhmän nimi,Seutukunta,Seutukunnan nimi,Maakunta,...,Suuralue,Suuralueen nimi,AVI,AVI:n nimi,ELY,ELY:n nimi,Hyvinvointialue,Hyvinvointialueen nimi,Vaalipiiri,Vaalipiirin nimi
0,100,Helsinki keskusta - Etu-Töölö,Helsingfors centrum - Främre Tölö,91,Helsinki,1,Kaupunkimaiset kunnat,11,Helsingin seutukunta,1,...,1,Helsinki-Uusimaa,1,Etelä-Suomen AVI,1,Uudenmaan ELY-keskus,90,Helsingin kaupunki,1,Helsingin vaalipiiri
1,120,Punavuori - Bulevardi,Rödbergen - Bulevarden,91,Helsinki,1,Kaupunkimaiset kunnat,11,Helsingin seutukunta,1,...,1,Helsinki-Uusimaa,1,Etelä-Suomen AVI,1,Uudenmaan ELY-keskus,90,Helsingin kaupunki,1,Helsingin vaalipiiri
2,130,Kaartinkaupunki,Gardesstaden,91,Helsinki,1,Kaupunkimaiset kunnat,11,Helsingin seutukunta,1,...,1,Helsinki-Uusimaa,1,Etelä-Suomen AVI,1,Uudenmaan ELY-keskus,90,Helsingin kaupunki,1,Helsingin vaalipiiri
3,140,Kaivopuisto - Ullanlinna,Brunnsparken - Ulrikasborg,91,Helsinki,1,Kaupunkimaiset kunnat,11,Helsingin seutukunta,1,...,1,Helsinki-Uusimaa,1,Etelä-Suomen AVI,1,Uudenmaan ELY-keskus,90,Helsingin kaupunki,1,Helsingin vaalipiiri
4,150,Punavuori - Eira - Hernesaari,Rödbergen - Eira - Ärtholmen,91,Helsinki,1,Kaupunkimaiset kunnat,11,Helsingin seutukunta,1,...,1,Helsinki-Uusimaa,1,Etelä-Suomen AVI,1,Uudenmaan ELY-keskus,90,Helsingin kaupunki,1,Helsingin vaalipiiri


In [22]:
postal_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3026 entries, 0 to 3025
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Postinumeroalue              3026 non-null   int64 
 1   Postinumeroalueen nimi       3026 non-null   object
 2   Postinumeroalueen nimi (sv)  3026 non-null   object
 3   Kunta                        3026 non-null   int64 
 4   Kunnan nimi                  3026 non-null   object
 5   Kuntaryhmä                   3026 non-null   int64 
 6   Kuntaryhmän nimi             3026 non-null   object
 7   Seutukunta                   3026 non-null   int64 
 8   Seutukunnan nimi             3026 non-null   object
 9   Maakunta                     3026 non-null   int64 
 10  Maakunnan nimi               3026 non-null   object
 11  Suuralue                     3026 non-null   int64 
 12  Suuralueen nimi              3026 non-null   object
 13  AVI                          3026

##### Select & rename columns

In [23]:
postal_df = postal_raw_df[[
    'Postinumeroalue', 
    'Postinumeroalueen nimi', 
    'Kunta', 
    'Kunnan nimi', 
    'Kuntaryhmä',
    'Kuntaryhmän nimi',
    'Maakunta',
    'Maakunnan nimi',
    'Suuralue',
    'Suuralueen nimi'
]].copy()

postal_df.rename(
    columns={
        'Postinumeroalue': 'Postal code',
        'Postinumeroalueen nimi': 'Postal code name', 
        'Kunta': 'Municipality', 
        'Kunnan nimi': 'Municipality name', 
        'Kuntaryhmä': 'Municipality group',
        'Kuntaryhmän nimi': 'Municipality group name',
        'Maakunta': 'Region',
        'Maakunnan nimi': 'Region name',
        'Suuralue': 'Area',
        'Suuralueen nimi': 'Area name'
    },
    inplace=True
)

postal_df.head()

,Postal code,Postal code name,Municipality,Municipality name,Municipality group,Municipality group name,Region,Region name,Area,Area name
0,100,Helsinki keskusta - Etu-Töölö,91,Helsinki,1,Kaupunkimaiset kunnat,1,Uusimaa,1,Helsinki-Uusimaa
1,120,Punavuori - Bulevardi,91,Helsinki,1,Kaupunkimaiset kunnat,1,Uusimaa,1,Helsinki-Uusimaa
2,130,Kaartinkaupunki,91,Helsinki,1,Kaupunkimaiset kunnat,1,Uusimaa,1,Helsinki-Uusimaa
3,140,Kaivopuisto - Ullanlinna,91,Helsinki,1,Kaupunkimaiset kunnat,1,Uusimaa,1,Helsinki-Uusimaa
4,150,Punavuori - Eira - Hernesaari,91,Helsinki,1,Kaupunkimaiset kunnat,1,Uusimaa,1,Helsinki-Uusimaa


##### Write to .csv

In [24]:
file_path = 'preprocessed_data/preprocessed_postal_codes.csv'
postal_df.to_csv(file_path, index=False)

# Merge apartment data

In [25]:
merged_apartment_df = pd.merge(rentals_df, sales_df, on=['Year', 'Postal code', 'Rooms'], how='outer')

merged_apartment_df.head()

,Year,Postal code,Rooms,Rent per m2,Rental apartment count,Sales price per m2,Sold apartments count
0,2015,100,1,24.7325,1820.0,6495.0,101.0
1,2015,100,2,18.3975,993.0,6313.0,181.0
2,2015,100,3,14.1425,156.0,5631.0,141.0
3,2015,120,1,22.6525,410.0,6755.0,42.0
4,2015,120,2,14.8325,242.0,6372.0,77.0


In [26]:
merged_apartment_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20863 entries, 0 to 20862
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   Year                    20863 non-null  int64   
 1   Postal code             20863 non-null  int64   
 2   Rooms                   20863 non-null  category
 3   Rent per m2             9904 non-null   float64 
 4   Rental apartment count  9904 non-null   float64 
 5   Sales price per m2      18233 non-null  float64 
 6   Sold apartments count   18233 non-null  float64 
dtypes: category(1), float64(4), int64(2)
memory usage: 1.1 MB


In [27]:
merged_apartment_df = pd.merge(merged_apartment_df, postal_df, on='Postal code', how='inner')

merged_apartment_df.head()

,Year,Postal code,Rooms,Rent per m2,Rental apartment count,Sales price per m2,Sold apartments count,Postal code name,Municipality,Municipality name,Municipality group,Municipality group name,Region,Region name,Area,Area name
0,2015,100,1,24.7325,1820.0,6495.0,101.0,Helsinki keskusta - Etu-Töölö,91,Helsinki,1,Kaupunkimaiset kunnat,1,Uusimaa,1,Helsinki-Uusimaa
1,2015,100,2,18.3975,993.0,6313.0,181.0,Helsinki keskusta - Etu-Töölö,91,Helsinki,1,Kaupunkimaiset kunnat,1,Uusimaa,1,Helsinki-Uusimaa
2,2015,100,3,14.1425,156.0,5631.0,141.0,Helsinki keskusta - Etu-Töölö,91,Helsinki,1,Kaupunkimaiset kunnat,1,Uusimaa,1,Helsinki-Uusimaa
3,2016,100,1,25.2650,2116.0,5780.0,167.0,Helsinki keskusta - Etu-Töölö,91,Helsinki,1,Kaupunkimaiset kunnat,1,Uusimaa,1,Helsinki-Uusimaa
4,2016,100,2,19.1600,1098.0,6138.0,157.0,Helsinki keskusta - Etu-Töölö,91,Helsinki,1,Kaupunkimaiset kunnat,1,Uusimaa,1,Helsinki-Uusimaa


In [28]:
merged_apartment_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20863 entries, 0 to 20862
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   Year                     20863 non-null  int64   
 1   Postal code              20863 non-null  int64   
 2   Rooms                    20863 non-null  category
 3   Rent per m2              9904 non-null   float64 
 4   Rental apartment count   9904 non-null   float64 
 5   Sales price per m2       18233 non-null  float64 
 6   Sold apartments count    18233 non-null  float64 
 7   Postal code name         20863 non-null  object  
 8   Municipality             20863 non-null  int64   
 9   Municipality name        20863 non-null  object  
 10  Municipality group       20863 non-null  int64   
 11  Municipality group name  20863 non-null  object  
 12  Region                   20863 non-null  int64   
 13  Region name              20863 non-null  object  
 14  Area  

##### Write to .csv

In [29]:
file_path = 'preprocessed_data/merged_apartment_data.csv'
merged_apartment_df.to_csv(file_path, index=False)

# Preprocessing of population & population forecast data

## Population data

##### Read .csv-file

In [30]:
csv_file = 'raw_data/11s1_keskiväkiluku.csv'

population_raw_df = pd.read_csv(csv_file, delimiter=';', encoding='latin1')

population_raw_df.head()

,Alue,Vuosi,Yhteensä Keskiväkiluku,0 - 14 Keskiväkiluku,15 - 24 Keskiväkiluku,25 - 44 Keskiväkiluku,45 - 64 Keskiväkiluku,65 - 74 Keskiväkiluku,75 - Keskiväkiluku
0,Akaa,1981,15768.5,3098.5,2392.0,4578.0,3522.0,1449.0,729.0
1,Akaa,1982,15752.0,3081.0,2335.5,4635.5,3497.5,1432.0,770.5
2,Akaa,1983,15727.5,3045.5,2309.0,4673.0,3482.5,1402.0,815.5
3,Akaa,1984,15731.0,3019.0,2283.0,4723.5,3480.0,1360.0,865.5
4,Akaa,1985,15696.0,2977.0,2255.0,4741.0,3478.0,1331.5,913.5


##### Rename columns

In [31]:
population_raw_df.rename(
    columns={
        'Alue': 'Municipality name', 
        'Vuosi': 'Year',
        'Yhteensä Keskiväkiluku': 'Total population',
        '0 - 14 Keskiväkiluku': '0 - 14',
        '15 - 24 Keskiväkiluku': '15 - 24',
        '25 - 44 Keskiväkiluku': '25 - 44',
        '45 - 64 Keskiväkiluku': '45 - 64',
        '65 - 74 Keskiväkiluku': '65 - 74',
        '75 - Keskiväkiluku': '75 -',
    },
    inplace=True
)

population_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12978 entries, 0 to 12977
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Municipality name  12978 non-null  object 
 1   Year               12978 non-null  int64  
 2   Total population   12978 non-null  float64
 3   0 - 14             12978 non-null  float64
 4   15 - 24            12978 non-null  float64
 5   25 - 44            12978 non-null  float64
 6   45 - 64            12978 non-null  float64
 7   65 - 74            12978 non-null  float64
 8   75 -               12978 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 912.6+ KB


##### Write to .csv

In [32]:
file_path = 'preprocessed_data/preprocessed_population.csv'
population_raw_df.to_csv(file_path, index=False)

## Population forecast data

##### Read .csv-file

In [33]:
csv_file = 'raw_data/139f_väestöennuste.csv'

population_fc_raw_df = pd.read_csv(csv_file, delimiter=';', encoding='latin1')

population_fc_raw_df.head()

,Alue,Vuosi,Yhteensä Väestö 31.12. (ennuste 2021),0 - 14 Väestö 31.12. (ennuste 2021),15 - 24 Väestö 31.12. (ennuste 2021),25 - 44 Väestö 31.12. (ennuste 2021),45 - 64 Väestö 31.12. (ennuste 2021),65 - 74 Väestö 31.12. (ennuste 2021),75 - Väestö 31.12. (ennuste 2021)
0,Akaa,2021,16264,2628,1479,3621,4482,2368,1686
1,Akaa,2022,16142,2519,1489,3534,4468,2357,1775
2,Akaa,2023,16023,2438,1473,3448,4478,2283,1903
3,Akaa,2024,15906,2356,1451,3394,4461,2244,2000
4,Akaa,2025,15795,2267,1446,3314,4477,2215,2076


##### Rename columns

In [34]:
population_raw_df.rename(
    columns={
        'Alue': 'Municipality name', 
        'Vuosi': 'Year',
        'Yhteensä Väestö 31.12. (ennuste 2021)': 'Total population',
        '0 - 14 Väestö 31.12. (ennuste 2021)': '0 - 14',
        '15 - 24 Väestö 31.12. (ennuste 2021)': '15 - 24',
        '25 - 44 Väestö 31.12. (ennuste 2021)': '25 - 44',
        '45 - 64 Väestö 31.12. (ennuste 2021)': '45 - 64',
        '65 - 74 Väestö 31.12. (ennuste 2021)': '65 - 74',
        '75 - Väestö 31.12. (ennuste 2021)': '75 -',
    },
    inplace=True
)

population_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12978 entries, 0 to 12977
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Municipality name  12978 non-null  object 
 1   Year               12978 non-null  int64  
 2   Total population   12978 non-null  float64
 3   0 - 14             12978 non-null  float64
 4   15 - 24            12978 non-null  float64
 5   25 - 44            12978 non-null  float64
 6   45 - 64            12978 non-null  float64
 7   65 - 74            12978 non-null  float64
 8   75 -               12978 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 912.6+ KB


##### Write to .csv

In [35]:
file_path = 'preprocessed_data/preprocessed_population_fc.csv'
population_raw_df.to_csv(file_path, index=False)

## Merging apartment & population data

In [36]:
merged_apartment_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20863 entries, 0 to 20862
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   Year                     20863 non-null  int64   
 1   Postal code              20863 non-null  int64   
 2   Rooms                    20863 non-null  category
 3   Rent per m2              9904 non-null   float64 
 4   Rental apartment count   9904 non-null   float64 
 5   Sales price per m2       18233 non-null  float64 
 6   Sold apartments count    18233 non-null  float64 
 7   Postal code name         20863 non-null  object  
 8   Municipality             20863 non-null  int64   
 9   Municipality name        20863 non-null  object  
 10  Municipality group       20863 non-null  int64   
 11  Municipality group name  20863 non-null  object  
 12  Region                   20863 non-null  int64   
 13  Region name              20863 non-null  object  
 14  Area  

In [37]:
grouped_by_municipality = merged_apartment_df.groupby(['Year', 'Municipality name', 'Rooms'], as_index=False)
grouped_by_municipality_df = grouped_by_municipality.agg({
    'Year': 'first',
    'Rooms': 'first',
    'Rent per m2': 'mean',
    'Rental apartment count': 'sum',
    'Sales price per m2': 'mean',
    'Sold apartments count': 'sum',
    'Municipality name': 'first',
    'Municipality group name': 'first',
    'Region name': 'first',
    'Area name': 'first'
})

grouped_by_municipality_df = grouped_by_municipality_df[(grouped_by_municipality_df['Sold apartments count'] != 0)].reset_index(drop=True)

grouped_by_municipality_df.head()

,Year,Rooms,Rent per m2,Rental apartment count,Sales price per m2,Sold apartments count,Municipality name,Municipality group name,Region name,Area name
0,2009.0,1,NaN,0.0,1131.0,20.0,Akaa,Taajaan asutut kunnat,Pirkanmaa,Länsi-Suomi
1,2009.0,2,NaN,0.0,1001.0,46.0,Akaa,Taajaan asutut kunnat,Pirkanmaa,Länsi-Suomi
2,2009.0,3,NaN,0.0,961.5,36.0,Akaa,Taajaan asutut kunnat,Pirkanmaa,Länsi-Suomi
3,2009.0,3,NaN,0.0,1434.0,8.0,Alajärvi,Taajaan asutut kunnat,Etelä-Pohjanmaa,Länsi-Suomi
4,2009.0,2,NaN,0.0,1024.0,11.0,Asikkala,Taajaan asutut kunnat,Päijät-Häme,Etelä-Suomi


In [38]:
merged_df = pd.merge(grouped_by_municipality_df, population_raw_df, on=['Year', 'Municipality name'], how='inner')

merged_df.head()

,Year,Rooms,Rent per m2,Rental apartment count,Sales price per m2,Sold apartments count,Municipality name,Municipality group name,Region name,Area name,Total population,0 - 14,15 - 24,25 - 44,45 - 64,65 - 74,75 -
0,2009.0,1,NaN,0.0,1131.0,20.0,Akaa,Taajaan asutut kunnat,Pirkanmaa,Länsi-Suomi,16847.5,3164.0,1666.5,4302.0,4810.0,1456.5,1448.5
1,2009.0,2,NaN,0.0,1001.0,46.0,Akaa,Taajaan asutut kunnat,Pirkanmaa,Länsi-Suomi,16847.5,3164.0,1666.5,4302.0,4810.0,1456.5,1448.5
2,2009.0,3,NaN,0.0,961.5,36.0,Akaa,Taajaan asutut kunnat,Pirkanmaa,Länsi-Suomi,16847.5,3164.0,1666.5,4302.0,4810.0,1456.5,1448.5
3,2009.0,3,NaN,0.0,1434.0,8.0,Alajärvi,Taajaan asutut kunnat,Etelä-Pohjanmaa,Länsi-Suomi,10603.5,1981.5,1317.0,2015.5,3143.0,1027.5,1119.0
4,2009.0,2,NaN,0.0,1024.0,11.0,Asikkala,Taajaan asutut kunnat,Päijät-Häme,Etelä-Suomi,8577.5,1311.0,803.0,1668.5,2860.0,1048.5,886.5


In [39]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4528 entries, 0 to 4527
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   Year                     4528 non-null   float64 
 1   Rooms                    4528 non-null   category
 2   Rent per m2              2091 non-null   float64 
 3   Rental apartment count   4528 non-null   float64 
 4   Sales price per m2       4528 non-null   float64 
 5   Sold apartments count    4528 non-null   float64 
 6   Municipality name        4528 non-null   object  
 7   Municipality group name  4528 non-null   object  
 8   Region name              4528 non-null   object  
 9   Area name                4528 non-null   object  
 10  Total population         4528 non-null   float64 
 11  0 - 14                   4528 non-null   float64 
 12  15 - 24                  4528 non-null   float64 
 13  25 - 44                  4528 non-null   float64 
 14  45 - 64 

##### Write to .csv

In [40]:
file_path = 'preprocessed_data/preprocessed_merged_pop_apartment.csv'
merged_df.to_csv(file_path, index=False)